### Setup

apt install requirements

In [1]:
# !apt-get install -y \
#     libgl1-mesa-dev \
#     libgl1-mesa-glx \
#     libglew-dev \
#     libosmesa6-dev \
#     software-properties-common

# !apt-get install -y patchelf

Clone repo and install dependencies

In [2]:
# %cd /content

# ## TODO: use when repo is public
# # !git clone https://github.com/jannerm/diffuser.git
# ## backup for private repo
# !curl -L -o diffuser.zip https://www.dropbox.com/s/fe1scd8m2jf7zvj/diffuser.tar?dl=1
# !tar -xf diffuser.zip
# ## /TODO

# %cd diffuser
# %pip install -f https://download.pytorch.org/whl/torch_stable.html \
#                 typed-argument-parser \
#                 scikit-image==0.17.2 \
#                 scikit-video==1.1.11 \
#                 gitpython \
#                 einops \
#                 pillow \
#                 free-mujoco-py \
#                 gym \
#                 git+https://github.com/JannerM/d4rl.git@0e84a4d29db3ae501043215ce1d91843929f1949 \
#                 git+https://github.com/aravindr93/mjrl

# %pip install -e .

In [3]:
import os
import json
import numpy as np
from os.path import join
import pdb

from diffuser.guides.policies import Policy
import diffuser.datasets as datasets
import diffuser.utils as utils

/opt/conda/envs/diffuser/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pybullet build time: Nov 28 2023 23:51:11


In [4]:
os.chdir('/root/diffuser')

Arguments

In [5]:
savepath = 'logs/maze2d-large-v1/rollouts'
os.makedirs(savepath, exist_ok=True)
vis_freq = 10
batch_size = 1

Loading

In [6]:
dataset = 'maze2d-large-v1'

env = datasets.load_environment(dataset)

diffusion_experiment = utils.load_diffusion('logs_2', dataset, 'diffusion/H384_T256', epoch='latest')

diffusion = diffusion_experiment.ema
dataset = diffusion_experiment.dataset
renderer = diffusion_experiment.renderer

policy = Policy(diffusion, dataset.normalizer)

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl

[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/render_config.pkl

[utils/config ] Config: <class 'diffuser.utils.rendering.Maze2dRenderer'>
    env: maze2d-large-v1

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/model_config.pkl

[utils/config ] Config: <class 'diffuser.models.temporal.TemporalUnet'>
    cond_dim: 4
    dim_mults: (1, 4, 8)
    horizon: 384
    transition_dim: 6

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/diffusion_config.pkl

[utils/config ] Config: <class 'diffuser.models.diffusion.GaussianDiffusion'>
  

load datafile: 100%|██████████| 8/8 [00:01<00:00,  7.19it/s]


[ utils/preprocessing ] Segmented maze2d-large-v1 | 1061 paths | min length: 67 | max length: 30470
[ datasets/buffer ] Finalized replay buffer | 1062 episodes
[ datasets/buffer ] Fields:
    actions: (1062, 40000, 2)
    infos/goal: (1062, 40000, 2)
    infos/qpos: (1062, 40000, 2)
    infos/qvel: (1062, 40000, 2)
    observations: (1062, 40000, 4)
    rewards: (1062, 40000, 1)
    terminals: (1062, 40000, 1)
    timeouts: (1062, 40000, 1)
    next_observations: (1062, 40000, 4)
    normed_observations: (1062, 40000, 4)
    normed_actions: (1062, 40000, 2)
[ models/temporal ] Channel dimensions: [(6, 32), (32, 128), (128, 256)]
[(6, 32), (32, 128), (128, 256)]

[ utils/serialization ] Loading model epoch: 1960000



In [7]:
dataset.normalizer

actions: [ Normalizer ] dim: 2
    -: [-1. -1.]
    +: [1. 1.]
]
infos/goal: [ Normalizer ] dim: 2
    -: [0.9 0.9]
    +: [ 7.1 10.1]
]
infos/qpos: [ Normalizer ] dim: 2
    -: [0.4  0.44]
    +: [ 7.22 10.22]
]
infos/qvel: [ Normalizer ] dim: 2
    -: [-5.23 -5.23]
    +: [5.23 5.23]
]
observations: [ Normalizer ] dim: 4
    -: [ 0.4   0.44 -5.23 -5.23]
    +: [ 7.22 10.22  5.23  5.23]
]
rewards: [ Normalizer ] dim: 1
    -: [0.]
    +: [1.]
]
terminals: [ Normalizer ] dim: 1
    -: [0.]
    +: [0.]
]
timeouts: [ Normalizer ] dim: 1
    -: [0.]
    +: [0.]
]
next_observations: [ Normalizer ] dim: 4
    -: [ 0.4   0.44 -5.23 -5.23]
    +: [ 7.22 10.22  5.23  5.23]
]

Main Loop

In [8]:
from tqdm import tqdm

In [29]:
observation = env.reset()
isconditioned = True

if isconditioned:
    print('Resetting target')
    env.set_target()

## set conditioning xy position to be the goal
target = env._target
cond = {
    diffusion.horizon - 1: np.array([*target, 0, 0]),
}

## observations for rendering
rollout = [observation.copy()]

total_reward = 0
scorelist = []
for i in tqdm(range(1)):
    total_reward = 0
    for t in range(env.max_episode_steps):

        state = env.state_vector().copy()

        ## can replan if desired, but the open-loop plans are good enough for maze2d
        ## that we really only need to plan once
        if t == 0:
            cond[0] = observation

            action, samples = policy(cond, batch_size=batch_size)
            actions = samples.actions[0]
            sequence = samples.observations[0]
        # pdb.set_trace()

        # ####
        if t < len(sequence) - 1:
            next_waypoint = sequence[t+1]
        else:
            next_waypoint = sequence[-1].copy()
            next_waypoint[2:] = 0
            # pdb.set_trace()

        ## can use actions or define a simple controller based on state predictions
        action = next_waypoint[:2] - state[:2] + (next_waypoint[2:] - state[2:])
        # pdb.set_trace()
        ####

        # else:
        #     actions = actions[1:]
        #     if len(actions) > 1:
        #         action = actions[0]
        #     else:
        #         # action = np.zeros(2)
        #         action = -state[2:]
        #         pdb.set_trace()



        next_observation, reward, terminal, _ = env.step(action)
        total_reward += reward
        score = env.get_normalized_score(total_reward)
        print(
            f't: {t} | r: {reward:.2f} |  R: {total_reward:.2f} | score: {score:.4f} | ref_max_score: {env.ref_max_score} | ref_min_score: {env.ref_min_score}'
            f'{action}'
        )

        xy = next_observation[:2]
        goal = env.unwrapped._target
        print(f'maze | pos: {xy} | goal: {goal}')

        ## update rollout observations
        rollout.append(next_observation.copy())

        # logger.log(score=score, step=t)

        if t % vis_freq == 0 or terminal:
            fullpath = join(savepath, f'{t}.png')

            if t == 0: renderer.composite(fullpath, samples.observations, ncol=1)



            ## save rollout thus far
            renderer.composite(join(savepath, 'rollout.png'), np.array(rollout)[None], ncol=1)



        if terminal:
            break

        observation = next_observation
    scorelist.append((total_reward, score)) 

Resetting target


  0%|          | 0/1 [00:00<?, ?it/s]


                                                                                                    
1 / 256 [                                                            ]   0% | 104.8 Hz
t : 255
                                                                                                    
                                                                                                    
2 / 256 [                                                            ]   0% | 113.3 Hz
t : 254
                                                                                                    
                                                                                                    
3 / 256 [                                                            ]   1% | 116.9 Hz
t : 253
                                                                                                    
                                                                                                    
4 / 256

100%|██████████| 1/1 [00:10<00:00, 10.05s/it]

Saved 1 samples to: logs/maze2d-large-v1/rollouts/rollout.png
sparse, mess it up
1.0, sparse, sparse is getting called
1.0, sparse, ... is getting called
t: 791 | r: 1.00 |  R: 240.00 | score: 0.8728 | ref_max_score: 273.99 | ref_min_score: 6.7[-0.0005148   0.00137548]
maze | pos: [6.92032884 3.96527563] | goal: [6.93527105 3.92535154]
sparse, mess it up
1.0, sparse, sparse is getting called
1.0, sparse, ... is getting called
t: 792 | r: 1.00 |  R: 241.00 | score: 0.8766 | ref_max_score: 273.99 | ref_min_score: 6.7[-0.00050953  0.00136141]
maze | pos: [6.92048178 3.964867  ] | goal: [6.93527105 3.92535154]
sparse, mess it up
1.0, sparse, sparse is getting called
1.0, sparse, ... is getting called
t: 793 | r: 1.00 |  R: 242.00 | score: 0.8803 | ref_max_score: 273.99 | ref_min_score: 6.7[-0.00050432  0.00134747]
maze | pos: [6.92063315 3.96446255] | goal: [6.93527105 3.92535154]
sparse, mess it up
1.0, sparse, sparse is getting called
1.0, sparse, ... is getting called
t: 794 | r: 1.00 |

In [10]:
scorelist

[(0.0, -0.02506640727299936)]

In [11]:
print("logs/pretrained/maze2d-large-v1")
np.mean([s[0] for s in scorelist]), np.std([s[0] for s in scorelist]), np.mean([s[1]*100 for s in scorelist]), np.std([s[1]*100 for s in scorelist])

logs/pretrained/maze2d-large-v1


(0.0, 0.0, -2.5066407272999363, 0.0)

In [12]:
print("logs/maze2d-large-v1")
np.mean([s[0] for s in scorelist]), np.std([s[0] for s in scorelist]), np.mean([s[1]*100 for s in scorelist]), np.std([s[1]*100 for s in scorelist])

logs/maze2d-large-v1


(0.0, 0.0, -2.5066407272999363, 0.0)

In [13]:
print("logs_2/maze2d-large-v1")
np.mean([s[0] for s in scorelist]), np.std([s[0] for s in scorelist]), np.mean([s[1]*100 for s in scorelist]), np.std([s[1]*100 for s in scorelist])

logs_2/maze2d-large-v1


(0.0, 0.0, -2.5066407272999363, 0.0)

In [14]:
np.std([s[1] for s in scorelist])

0.0

In [15]:
json_path = join(savepath, 'rollout.json')
json_data = {'score': score, 'step': t, 'return': total_reward, 'term': terminal,
    'epoch_diffusion': diffusion_experiment.epoch}
json.dump(json_data, open(json_path, 'w'), indent=2, sort_keys=True)

In [16]:
json_data

{'score': -0.02506640727299936,
 'step': 799,
 'return': 0.0,
 'term': False,
 'epoch_diffusion': 1960000}

In [17]:
json_data

{'score': -0.02506640727299936,
 'step': 799,
 'return': 0.0,
 'term': False,
 'epoch_diffusion': 1960000}

In [18]:
print('Ref min score:', env.ref_min_score, 'Ref max score:', env.ref_max_score)

Ref min score: 6.7 Ref max score: 273.99


In [19]:
json_data

{'score': -0.02506640727299936,
 'step': 799,
 'return': 0.0,
 'term': False,
 'epoch_diffusion': 1960000}

In [20]:
100*(439.0-env.ref_min_score)/(env.ref_max_score - env.ref_min_score)

161.73444573309888

In [21]:
print('Ref min score:', env.ref_min_score, 'Ref max score:', env.ref_max_score)

Ref min score: 6.7 Ref max score: 273.99


In [22]:
print('Ref min score:', env.ref_min_score, 'Ref max score:', env.ref_max_score)

Ref min score: 6.7 Ref max score: 273.99


In [23]:
import json
import numpy as np

In [24]:
metrics = {}

In [25]:
datasets = ['maze2d-large-v1', 'maze2d-medium-v1', 'maze2d-umaze-v1']

In [26]:
for dataset in datasets:
    json_path = f'/root/diffuser/logs/pretrained/{dataset}/rollouts/scorelist.json'
    with open(json_path, 'r') as f:
        data = json.load(f)
    scores = np.array(data['scores'])
    metrics.update({dataset: {'mean': np.mean(scores), 'std': np.std(scores)}})


In [27]:
with open('/root/diffuser/logs/pretrained/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

In [28]:
dataset = 'maze2d-large-v1'
json_path = f'/root/diffuser/logs_2/{dataset}/rollouts/scorelist.json'
with open(json_path, 'r') as f:
    data = json.load(f)
scores = np.array(data['scores'])

FileNotFoundError: [Errno 2] No such file or directory: '/root/diffuser/logs_2/maze2d-large-v1/rollouts/scorelist.json'

In [ ]:
scores

NameError: name 'scores' is not defined

In [ ]:
np.mean(scores), np.std(scores)

NameError: name 'np' is not defined

In [ ]:
import pickle

# Specify the path to the pickle file
pkl_file_path = "logs/pretrained/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl"

# Load the pickle file
with open(pkl_file_path, "rb") as f:
    data = pickle.load(f)

# Now you can use the loaded data
print(data)


[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False



In [ ]:
dataset_config = utils.Config(
    'datasets.GoalDataset',
    savepath='logs/pretrained/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl',
    env='maze2d-large-v1',
    horizon=384,
    normalizer='LimitsNormalizer',
    preprocess_fns=['maze2d_set_terminals'],
    use_padding=False,
    max_path_length=40000,
)

[ utils/config ] Imported diffuser.datasets:GoalDataset

[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False

[ utils/config ] Saved config to: logs/pretrained/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl



In [ ]:
dataset = dataset_config()

load datafile: 100%|██████████| 8/8 [00:01<00:00,  7.19it/s]


[ utils/preprocessing ] Segmented maze2d-large-v1 | 1061 paths | min length: 67 | max length: 30470
[ datasets/buffer ] Finalized replay buffer | 1062 episodes
[ datasets/buffer ] Fields:
    actions: (1062, 40000, 2)
    infos/goal: (1062, 40000, 2)
    infos/qpos: (1062, 40000, 2)
    infos/qvel: (1062, 40000, 2)
    observations: (1062, 40000, 4)
    rewards: (1062, 40000, 1)
    terminals: (1062, 40000, 1)
    timeouts: (1062, 40000, 1)
    next_observations: (1062, 40000, 4)
    normed_observations: (1062, 40000, 4)
    normed_actions: (1062, 40000, 2)


In [ ]:
import torch

In [ ]:
def cycle(dl):
    while True:
        for data in dl:
            yield data

In [ ]:
dataloader = cycle(torch.utils.data.DataLoader(
            dataset, batch_size=1, num_workers=1, shuffle=True, pin_memory=True
        ))

In [ ]:
dataloader

<generator object cycle at 0x7fe6d8a1fcf0>

In [ ]:
batch = next(dataloader)

In [ ]:
from arrays import batch_to_device, to_np, to_device, apply_dict


In [ ]:
batch = batch_to_device(batch)

In [ ]:
traj = to_np(batch.trajectories)

In [ ]:
traj

array([[[ 0.7690196 ,  0.1735717 ,  0.31130707, -0.71825343,
         -0.13593501,  0.3801254 ],
        [ 0.8850416 , -0.32477522,  0.30976582, -0.7141149 ,
         -0.10056651,  0.3871299 ],
        [ 0.76178956, -0.72983533,  0.30884624, -0.7101445 ,
         -0.05999506,  0.37140775],
        ...,
        [ 0.17779183, -1.        , -0.29482794,  0.0478698 ,
          0.01712537,  0.8765112 ],
        [ 0.37615836, -1.        , -0.29444194,  0.05673027,
          0.02518666,  0.82885325],
        [ 0.01288068, -0.37463444, -0.2937941 ,  0.06508267,
          0.04226851,  0.78130853]]], dtype=float32)

In [ ]:
conditions = to_np(batch.conditions[0])[:,None]

In [ ]:
conditions

array([[[ 0.870726  ,  0.14610803, -0.34367085,  0.10682023]]],
      dtype=float32)